In [1]:
!nvidia-smi

Sun Mar 17 21:34:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8               3W /  65W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y, Y.device

(tensor([[0.9132, 0.9618, 0.8861],
         [0.8011, 0.7350, 0.0369]]),
 device(type='cpu'))

In [8]:
Z = Y.cuda()
print(X)
print(Y)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[0.9132, 0.9618, 0.8861],
        [0.8011, 0.7350, 0.0369]])
tensor([[0.9132, 0.9618, 0.8861],
        [0.8011, 0.7350, 0.0369]], device='cuda:0')


In [9]:
Z + X

tensor([[1.9132, 1.9618, 1.8861],
        [1.8011, 1.7350, 1.0369]], device='cuda:0')

In [10]:
Z.cuda(0) is Z

True

In [11]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [12]:
net(X)

tensor([[0.9473],
        [0.9473]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)